In [13]:
import os

In [14]:
%pwd

'/Users/manasvialam/Documents/ML_projects/Kidney-Disease-Classification'

In [15]:
# change directory
os.chdir("../")

In [16]:
%pwd

'/Users/manasvialam/Documents/ML_projects'

In [19]:
os.chdir('/Users/manasvialam/Documents/ML_projects/Kidney-Disease-Classification')

In [20]:
%pwd

'/Users/manasvialam/Documents/ML_projects/Kidney-Disease-Classification'

In [21]:
# entity is the return type of function

from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class DataIngestionConfig:
    rootdir: Path
    source_URL : str
    local_data_file : Path
    unzip_dir: Path

In [22]:
from cnnClassifier.constant import *
from cnnClassifier.utils.common import read_yaml, create_directories

In [23]:
class ConfigurationManager:
    def __init__(
            self,
            config_filepath= CONFIG_FILE_PATH,
            params_filepath= PARAMS_FILE_PATH):

            self.config = read_yaml(config_filepath)
            self.params = read_yaml(params_filepath)

            create_directories([self.config.artifacts_root])

    def get_data_ingestion_config(self) -> DataIngestionConfig:
          config = self.config.data_ingestion

          create_directories([config.root_dir])

          data_ingestion_config = DataIngestionConfig(
                rootdir= config.root_dir,
                source_URL= config.source_URL,
                local_data_file= config.local_data_file,
                unzip_dir= config.unzip_dir
          )
          
          return data_ingestion_config
    

In [24]:
import os
import zipfile
import gdown
from cnnClassifier import logger
from cnnClassifier.utils.common import get_size

In [27]:
import gdown.download


class DataIngestion:
    def __init__(self, config: DataIngestionConfig):
        self.config = config

    def download_file(self)-> str:
        '''
        Fetch data from the url
        '''

        try: 
            dataset_url = self.config.source_URL
            zip_download_dir = self.config.local_data_file
            os.makedirs("artifacts/data_ingestion", exist_ok=True)
            logger.info(f"Downloading data from {dataset_url} into file {zip_download_dir}")

            file_id = dataset_url.split("/")[-2]
            prefix = 'https://drive.google.com/uc?/export=download&id='
            gdown.download(prefix+file_id, zip_download_dir)

            logger.info(f"Downloaded data from {dataset_url} into file {zip_download_dir}")

        except Exception as e:
            raise e
        
    def extract_zip_file(self):
        """
        zip_file_path: str
        Extracts the zip file into the data directory
        Function returns None
        """
        unzip_path = self.config.unzip_dir
        os.makedirs(unzip_path, exist_ok=True)
        with zipfile.ZipFile(self.config.local_data_file,'r') as zip_ref:
            zip_ref.extractall(unzip_path)

In [28]:
# Pipeline -> first we are downloading the file and then extracting it from the zip file
try:
    config= ConfigurationManager()
    data_ingestion_config = config.get_data_ingestion_config()
    data_ingestion = DataIngestion(config=data_ingestion_config)
    data_ingestion.download_file()
    data_ingestion.extract_zip_file()
except Exception as e:
    raise e

[2024-09-13 18:41:57,923:INFO: common: yaml file: config/config.yaml loaded successfully]
[2024-09-13 18:41:57,925:INFO: common: yaml file: params.yaml loaded successfully]
[2024-09-13 18:41:57,925:INFO: common: created directory at: artifacts]
[2024-09-13 18:41:57,926:INFO: common: created directory at: artifacts/data_ingestion]
[2024-09-13 18:41:57,927:INFO: 606855465: Downloading data from https://drive.google.com/file/d/1YKuwii_lywWI_03kOtWC8HF7xy1w2-93/view?usp=drive_link into file artifacts/data_ingestion/data.zip]


Downloading...
From (original): https://drive.google.com/uc?/export=download&id=1YKuwii_lywWI_03kOtWC8HF7xy1w2-93
From (redirected): https://drive.google.com/uc?%2Fexport=download&id=1YKuwii_lywWI_03kOtWC8HF7xy1w2-93&confirm=t&uuid=e9ee0a4e-740d-49ca-9f2a-2bb2086fec9c
To: /Users/manasvialam/Documents/ML_projects/Kidney-Disease-Classification/artifacts/data_ingestion/data.zip
100%|██████████| 73.5M/73.5M [00:06<00:00, 11.0MB/s]

[2024-09-13 18:42:09,812:INFO: 606855465: Downloaded data from https://drive.google.com/file/d/1YKuwii_lywWI_03kOtWC8HF7xy1w2-93/view?usp=drive_link into file artifacts/data_ingestion/data.zip]
